# sampley exemplar: Stage 3 (continuous)
Before going through this exemplar, please consult the Introduction to sampley exemplars (```intro.ipynb```).
<br>This exemplar illustrates the classes that can be made in Stage 3, their class methods, their attributes, and their methods.
<br>It differs from the Stage 3 exemplar in that it uses a single input file containing continuous datapoints (i.e., datapoints recorded at frequent, regular intervals and from which survey tracks can be made). If using the segment or point approach, having continuous datapoints allows for certain procedures to be applied that may be more efficient and accurate.

## Setup

### Import the package

In [1]:
from sampley import *

### Set the input folder
To run this exemplar, download the mock data files, put them in a folder, and set the path to the folder below.

In [2]:
input_folder = './input/'

### Set the output folder
To run this exemplar, make a folder to save the outputs in and set the path to the folder below.

In [3]:
output_folder = './output/'

### Make DataPoints, Sections, and delimiters (Stages 1 and 2)
Before making any objects in Stage 3, we, of course, have to conduct Stages 1 and 2. For the sake of this exemplar, we make one ```Sections``` and one ```DataPoints``` object below that are then used to make all the subsequent delimiters. See the Stage 1 and 2 exemplars and the User Manual for more details on Stages 1 and 2.

In [4]:
u_trackpoints = DataPoints.from_file(
    filepath=input_folder+'trackpoints.csv',
    x_col='lon',
    y_col='lat',
    crs_input='EPSG:4326',
    crs_working='EPSG:32619',
    datetime_col='datetime',
    tz_input='UTC-05:00',
    tz_working='UTC-05:00',
    section_id_col='section_id'  
)

Opening file...
Success: file successfully input.
Parsing x and y (lon/lat) coordinates...
Success: x and y (lon/lat) coordinates successfully parsed.
Reprojecting CRS...
Success: reprojected to CRS 'EPSG:32619'
Parsing datetimes...
Success: the column 'datetime' successfully reformatted to datetimes.
Success: the timezone of column 'datetime' successfully set to 'UTC-05:00'.
Converting timezone...
Note: conversion of column 'datetime' to timezone 'UTC-05:00' is not necessary as it is already in timezone 'UTC-05:00'.
Success: datapoint IDs successfully generated.


In [5]:
u_sections = Sections.from_datapoints(datapoints=u_trackpoints)

In [6]:
u_segments = Segments.delimit(
    sections=u_sections,
    var='simple',
    target=10000,
    randomise=True)

In [8]:
u_presences = Presences.delimit(
    datapoints=u_trackpoints,
    presence_col='individuals')
u_presences.thin(
    sp_threshold=10000,
    tm_threshold=5,
    tm_unit='day')

Conducting spatiotemporal thinning...
Thinning complete.


In [9]:
u_absencelines = AbsenceLines.delimit(
    sections=u_sections,
    presences=u_presences,
    sp_threshold=10000,
    tm_threshold=5,
    tm_unit='day',
)

Note: absence lines to be generated with a temporal threshold of 5 day(s).


In [10]:
u_absences = Absences.delimit(
    absencelines=u_absencelines,
    var='along',
    target=20,
    dfls=None)
u_absences.thin(
    sp_threshold=10000,
    tm_threshold=5,
    tm_unit='day',
    target=9)

Conducting spatiotemporal thinning...
Thinning complete.


## Samples - segment approach

### Make a ```Samples``` object...
If using the segment approach, we can make a ```Samples``` object from a ```DataPoints``` object with the ```Samples.segment()``` class method.
<br>We can also make a ```Samples``` object with measures of survey effort from a ```Segments``` object with the ```Samples.segment_se()``` class method.
<br>Additionally, we can merge multiple ```Samples``` objects into a single new ```Samples``` object with the ```Samples.merge()``` class method.

#### ...from a ```DataPoints``` object
As the segments are made from sections that are made from datapoints, we can match those datapoints to the segments by their distance from the beginning of the section (```how='dfb'```), rather than by absolute distance to the midpoint or line. This is more accurate and faster.

In [11]:
u_samples_trackpoints = Samples.segment(
    datapoints=u_trackpoints,
    segments=u_segments,
    cols={'individuals': 'sum', 'bss': 'mean'},
    how='dfb')

#### ...from a ```Sections``` object

In [12]:
u_samples_effort = Samples.segment_se(
    segments=u_segments,
    esw=2000)

#### ...by merging multiple ```Samples``` objects

In [13]:
u_samples = Samples.merge(
    sightings=u_samples_trackpoints,
    effort=u_samples_effort)


Note: samples generated with the segment approach


### Access a ```Samples``` object's attributes
A ```Samples``` object, regardless of how it was made, will have three attributes (```name```, ```parameters```, and ```samples```) that we can access as follows.

In [14]:
u_samples.name

'samples-sightings+effort-x-segments-s10000m'

In [15]:
u_samples.parameters

{'names': 'samples-datapoints-trackpoints-x-segments-s10000m+samples-sections-trackpoints-x-segments-s10000m',
 'approach': 'segment',
 'resampled': 'datapoints; effort',
 'datapoints_name': 'datapoints-trackpoints; nan',
 'datapoints_filepath': './input/trackpoints.csv; nan',
 'datapoints_crs': 'EPSG:32619; nan',
 'datapoints_tz': 'UTC-05:00; nan',
 'datapoints_data_cols': 'section_id, individuals, bss; nan',
 'segments_name': 'segments-s10000m',
 'sections_name': 'sections-trackpoints',
 'segments_crs': 'EPSG:32619',
 'segments_var': 'simple',
 'segments_randomise': 'True',
 'segments_target': '10000',
 'segments_unit': 'metre',
 'cols': "{'individuals': 'sum', 'bss': 'mean'}; nan",
 'effort_esw': 'nan; 2000.0',
 'effort_audf': 'nan; None',
 'effort_euc-geo': 'nan; euclidean'}

In [16]:
u_samples.samples

,segment_id,line,midpoint,date,section_id,dfbsec_beg,dfbsec_end,individuals,bss,se_length,se_area
0,s01-s10000m,"LINESTRING (580092.757 4742883.408, 579997.135...",POINT (575093.311 4742845.962),2019-01-25,s001,0.000000,10000.000000,1.0,2.000000,10000.00000,4.000000e+07
1,s02-s10000m,"LINESTRING (570094.222 4742829.916, 570089.58 ...",POINT (570091.901 4742829.892),2019-01-25,s001,10000.000000,10004.642260,NaN,NaN,4.64226,1.856904e+04
2,s03-s10000m,"LINESTRING (570089.58 4742829.868, 569917.081 ...",POINT (565090.088 4742799.681),2019-01-25,s001,10004.642260,20004.642260,0.0,2.000000,10000.00000,4.000000e+07
3,s04-s10000m,"LINESTRING (560090.506 4742773.123, 559864.339...",POINT (555090.879 4742749.082),2019-01-25,s001,20004.642260,30004.642260,2.0,2.000000,10000.00000,4.000000e+07
4,s05-s10000m,"LINESTRING (550091.025 4742710.901, 549838.842...",POINT (545091.144 4742676.352),2019-01-25,s001,30004.642260,40004.642260,5.0,2.846154,10000.00000,4.000000e+07
...,...,...,...,...,...,...,...,...,...,...,...
68,s69-s10000m,"LINESTRING (652338.76 4697682.02, 652197.878 4...",POINT (647339.987 4697611.472),2019-02-05,s004,274105.324868,284105.324868,0.0,2.000000,10000.00000,4.000000e+07
69,s70-s10000m,"LINESTRING (642341.15 4697526.723, 641994.147 ...",POINT (637342.194 4697445.552),2019-02-05,s004,284105.324868,294105.324868,0.0,2.814815,10000.00000,4.000000e+07
70,s71-s10000m,"LINESTRING (632343.125 4697349.07, 632326.134 ...",POINT (627344.065 4697254.461),2019-02-05,s004,294105.324868,304105.324868,0.0,3.000000,10000.00000,4.000000e+07
71,s72-s10000m,"LINESTRING (622345.629 4697147.973, 622000.67 ...",POINT (617346.364 4697062.225),2019-02-05,s004,304105.324868,314105.324868,2.0,3.000000,10000.00000,4.000000e+07


### Save a ```Samples``` object
```Samples``` objects have an inbuilt ```save``` method to save the samples as a CSV or GPKG.

In [17]:
u_samples.save(
    folder=output_folder,
    filetype='gpkg'
)

## Samples - point approach

### Make a ```Samples``` object...
If using the point approach, we can make a ```Samples``` object from a ```DataPoints``` object with the ```Samples.point()``` class method.

#### ...from a ```DataPoints``` object
As the absence lines are made from sections that are made from datapoints, we can match those datapoints to the absences by their distance from the beginning of the absences lines in order to give values to the absences. To do so we must input the sections (```sections=u_sections```).

In [18]:
u_samples = Samples.point(
    datapoints=u_trackpoints,
    presences=u_presences,
    absences=u_absences,
    cols=['individuals', 'bss'],
    sections=u_sections)

### Access a ```Samples``` object's attributes
A ```Samples``` object, regardless of how it was made, will have three attributes (```name```, ```parameters```, and ```samples```) that we can access as follows.

In [19]:
u_samples.name

'samples-presences-trackpoints-+-absences-a-10000m-5day'

In [20]:
u_samples.parameters

{'approach': 'point',
 'resampled': 'datapoints',
 'presences_name': 'presences-trackpoints',
 'presences_crs': 'EPSG:32619',
 'presences_sp_threshold': 10000,
 'presences_tm_threshold': 5,
 'presences_tm_unit': 'day',
 'absences_name': 'absences-a-10000m-5day',
 'absences_var': 'along',
 'absences_target': 20,
 'absences_crs': 'EPSG:32619',
 'absences_sp_threshold': 10000,
 'absences_tm_threshold': 5,
 'absences_tm_unit': 'day'}

In [21]:
u_samples.samples

,point_id,point,date,p-a,individuals,bss,datapoint_id
0,p01,POINT (579166.78 4742872.701),2019-01-25,1,1.0,2,d0004
1,p02,POINT (554184.217 4742741.886),2019-01-25,1,2.0,2,d0068
2,p04,POINT (520909.741 4714855.058),2019-02-02,1,1.0,1,d0480
3,p05,POINT (532548.249 4714899.835),2019-02-02,1,2.0,1,d0510
4,p06,POINT (512817.407 4705582.465),2019-02-02,1,1.0,1,d0910
5,p08,POINT (654449.136 4716189.584),2019-02-05,1,5.0,1,d1306
6,p10,POINT (643532.681 4716066.52),2019-02-05,1,1.0,2,d1336
7,p11,POINT (629124.489 4706545.106),2019-02-05,1,3.0,2,d1527
8,p13,POINT (611976.857 4696974.111),2019-02-05,1,4.0,3,d1739
9,a17,POINT (592247.065 4733695.898),2019-03-03,0,NaN,3,d0209


### Save a ```Samples``` object
```Samples``` objects have an inbuilt ```save``` method to save the samples as a CSV or GPKG.

In [22]:
u_samples.save(
    folder=output_folder,
    filetype='gpkg'
)